This notebook extracts data from the Wikipedia page on **Highest-Grossing Films** and stores it in a relational database. The extracted data includes film titles, release years, directors, box office revenue, and countries of origin.

# Step 1: Import Required Libraries

In this step, we import the necessary Python libraries for the task:
- `requests`: To send HTTP requests and fetch the content of the Wikipedia page.
- `BeautifulSoup` (from `bs4`): To parse the HTML content and extract the required data.
- `sqlite3`: To interact with the SQLite database where we will store the extracted data.

In [1]:
import requests
from bs4 import BeautifulSoup
import sqlite3

# Step 2: Fetch and Parse the Wikipedia Page

In this step, we:
1. Define the URL of the Wikipedia page containing the list of highest-grossing films.
2. Use the `requests.get()` method to send a GET request and fetch the HTML content of the page.
3. Parse the HTML content using `BeautifulSoup` to create a structured object (`soup`) that allows us to easily navigate and extract data from the page.

In [2]:
# URL of the Wikipedia page
url = "https://en.wikipedia.org/wiki/List_of_highest-grossing_films"

# Send a GET request to fetch the page content
response = requests.get(url)
html_content = response.text

# Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(html_content, "html.parser")

# Step 3: Extract Data from the Table

In this step, we:
1. Locate the HTML table containing the film data using the `soup.find()` method. The table has the class `wikitable`.
2. Initialize an empty list (`films_data`) to store the extracted data for each film.
3. Iterate through each row of the table (skipping the header row) using `table.find_all("tr")[1:]`.
4. For each row:
   - Extract the film title from the `<th>` element and its nested `<a>` tag. If the title is not found, use a fallback value (`"Unknown Title"`).
   - Extract the release year from the appropriate column.
   - Follow the reference link (`href`) in the title tag to the film's individual Wikipedia page to fetch additional details (director, country, and box office revenue).
   - Parse the reference page to locate and extract the director, country, and box office revenue.
5. Append the extracted data as a tuple to the `films_data` list.

In [3]:
# Find the table containing the film data
table = soup.find("table", {"class": "wikitable"})

films_data = []

# Iterate through each row in the table
for row in table.find_all("tr")[1:]:
    columns = row.find_all("td")

    # Locate the <th> element within the current row and find the <a> tag
    title_tag = row.find("th").find("a") if row.find("th") else None

    # Check if title_tag is not None before accessing .text
    if title_tag:
        title = title_tag.text.strip()
    else:
        title = "Unknown Title"  # Fallback in case the title is not found

    release_year = int(columns[3].text.strip())

    ref_link = title_tag["href"] if title_tag else None

    director = "Unknown Director"
    country = "Unknown Country"

    if ref_link:
        ref_url = f"https://en.wikipedia.org{ref_link}"
        ref_response = requests.get(ref_url)
        ref_soup = BeautifulSoup(ref_response.content, "html.parser")

        # Locate the director information on the reference page
        director_tag = ref_soup.find("th", string="Directed by")
        if director_tag:
            director = director_tag.find_next("td").text.strip()

        country_tag = ref_soup.find("th", string="Country") or ref_soup.find("th", string="Countries")
        if country_tag:
            country = country_tag.find_next("td").text.strip()

        box_office_tag = ref_soup.find("th", string="Box office")
        if box_office_tag:
            box_office = box_office_tag.find_next("td").text.strip()

    # Append the extracted data to the list
    films_data.append((title, release_year, director, box_office, country))

# Step 4: Data Cleaning Functions

In this step, we define helper functions to clean the extracted data:
1. **`clean_box_office(box_office)`**:
   - Removes currency symbols (e.g., `$`), footnotes, and non-numeric characters from the box office revenue.
   - Converts the cleaned value to a `float` for numerical analysis. If the value is empty, it returns `None`.

2. **`clean_director(director)`**:
   - Replaces newlines and multiple spaces with a single space for consistency.
   - Removes Wikipedia footnotes (e.g., `[1]`, `[2]`) from the director names.

3. **`clean_country(country)`**:
   - Removes Wikipedia footnotes (e.g., `[1]`, `[2]`) and newlines from the country names.
   - Fixes specific formatting issues (e.g., replacing `"United States China"` with `"United States, China"`).

It is done to ensure that the extracted data is consistent and ready for insertion into the database.

In [4]:
import re

# Function to clean box office revenue
def clean_box_office(box_office):
    cleaned_value = re.sub(r"[^\d.]", "", box_office)
    return float(cleaned_value) if cleaned_value else None

# Function to clean director names
def clean_director(director):
    cleaned_director = re.sub(r"\s+", " ", director.replace("\n", ", ")).strip()
    cleaned_director = re.sub(r"\[\d+\]", "", director.replace("\n", ", ")).strip()
    return cleaned_director

# Function to clean country names
def clean_country(country):
    cleaned_country = re.sub(r"\[\d+\]", "", country.replace("\n", ", ")).strip()
    cleaned_country = re.sub(r"United States China", r"United States, China", cleaned_country)
    cleaned_country = re.sub(r"United KingdomUnited States", r"United Kingdom, United States", cleaned_country)
    return cleaned_country

## Step 5: Clean and Structure the Data

In this step, we:
1. **Clean and Structure the Data**:
   - Iterate through the `films_data` list (which contains raw extracted data).
   - For each film, create a dictionary (`cleaned_film`) with cleaned and structured data.
   - Append each cleaned film dictionary to the `cleaned_films` list.

2. **Convert Box Office Revenue**:
   - Convert the box office revenue from billions (e.g., `1.5`) to its full numeric value (e.g., `1,500,000,000`).

In [5]:
# Clean and structure the data
cleaned_films = []
for film in films_data:
    cleaned_film = {
        "title": film[0].strip(),  # Clean title
        "release_year": int(film[1]),  # Convert release year to integer
        "director": clean_director(film[2]),  # Clean director names
        "box_office": clean_box_office(film[3]),  # Clean box office revenue
        "country": clean_country(film[4])  # Clean country names
    }
    cleaned_films.append(cleaned_film)

# Convert box office from billions to full numeric value
for film in cleaned_films:
    film["box_office"] = round(film["box_office"] * 1_000_000_000)  # Convert to full value

## Step 6: Store Data in SQLite Database

In this step, we:
1. **Connect to the SQLite Database**:
   - Use the `sqlite3` library to connect to a SQLite database file named `highest_grossing_films.db`. If the file doesn't exist, it will be created automatically.

2. **Create the `films` Table**:
   - Define a table schema with the following columns:
     - `id`: A unique identifier for each film (auto-incremented).
     - `title`: The title of the film (text, not null).
     - `release_year`: The year the film was released (integer).
     - `director`: The name(s) of the director(s) (text).
     - `box_office`: The box office revenue (real number).
     - `country`: The country of origin (text).
   - If the table already exists, it will not be recreated.

3. **Insert the Cleaned Data**:
   - Iterate through the `cleaned_films` list and insert each film's data into the `films` table using parameterized queries to prevent SQL injection.

4. **Commit and Close**:
   - Commit the transaction to save the changes to the database.
   - Close the database connection.

In [6]:
import sqlite3

# Connect to the SQLite database
conn = sqlite3.connect("highest_grossing_films.db")
cursor = conn.cursor()

# Create the films table (if it doesn't already exist)
cursor.execute("""
CREATE TABLE IF NOT EXISTS films (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    title TEXT NOT NULL,
    release_year INTEGER,
    director TEXT,
    box_office REAL,
    country TEXT
)
""")

# Insert the cleaned data into the table
for film in cleaned_films:
    cursor.execute("""
    INSERT INTO films (title, release_year, director, box_office, country)
    VALUES (?, ?, ?, ?, ?)
    """, (film["title"], film["release_year"], film["director"], film["box_office"], film["country"]))

# Commit the transaction and close the connection
conn.commit()
conn.close()

## Step 7: Verify Data Insertion

In this step, we:
1. **Reconnect to the Database**:
   - Reopen the SQLite database (`highest_grossing_films.db`) to verify that the data was inserted correctly.

2. **Fetch and Display Data**:
   - Execute a `SELECT` query to retrieve all rows from the `films` table.
   - Use `fetchall()` to fetch the results and print each row to the console.
   - This allows us to visually inspect the data.

3. **Close the Connection**:
   - Close the database connection.

In [7]:
# Reconnect to the database
conn = sqlite3.connect("highest_grossing_films.db")
cursor = conn.cursor()

# Fetch and display the films
cursor.execute("SELECT * FROM films")
rows = cursor.fetchall()
for row in rows:
    print(row)

# Close the connection
conn.close()

(1, 'Avatar', 2009, 'James Cameron', 2923500000.0, 'United Kingdom, United States')
(2, 'Avengers: Endgame', 2019, 'Anthony RussoJoe Russo', 2799400000.0, 'United States')
(3, 'Avatar: The Way of Water', 2022, 'James Cameron', 2320450000.0, 'United States')
(4, 'Titanic', 1997, 'James Cameron', 2264700000.0, 'United States')
(5, 'Star Wars: The Force Awakens', 2015, 'J. J. Abrams', 2073000000.0, 'United States')
(6, 'Avengers: Infinity War', 2018, 'Anthony RussoJoe Russo', 2052400000.0, 'United States')
(7, 'Ne Zha 2', 2025, 'Jiaozi', 1982300000.0, 'China')
(8, 'Spider-Man: No Way Home', 2021, 'Jon Watts', 1923340000.0, 'United States')
(9, 'Inside Out 2', 2024, 'Kelsey Mann', 1699340000.0, 'United States')
(10, 'Jurassic World', 2015, 'Colin Trevorrow', 1671400000.0, 'United States')
(11, 'The Lion King', 2019, 'Jon Favreau', 1657500000.0, 'United States')
(12, 'The Avengers', 2012, 'Joss Whedon', 1521400000.0, 'United States')
(13, 'Furious 7', 2015, 'James Wan', 1515400000.0, 'Unite

## Export Database to JSON

This Python script connects to an SQLite database, fetches film data, and exports it to a JSON file. The JSON file will be used to populate the web page.

In [8]:
import sqlite3
import json

# Connect to the SQLite database
conn = sqlite3.connect("highest_grossing_films.db")
cursor = conn.cursor()

# Fetch all rows from the films table
cursor.execute("SELECT * FROM films")
rows = cursor.fetchall()

# Convert rows to a list of dictionaries
films_data = []
for row in rows:
    films_data.append({
        "title": row[1],
        "release_year": row[2],
        "director": row[3],
        "box_office": row[4],
        "country": row[5]
    })

# Save the data to a JSON file
with open("data_1.json", "w") as json_file:
    json.dump(films_data, json_file, indent=4)

# Close the connection
conn.close()